In [9]:
pip install keras


     |████████████████████████████████| 27.2 MB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 662 kB 10.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
# import tf as tf

if tf.test.gpu_device_name():

    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")

tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Default GPU Device: /device:GPU:0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [2]:

#!/usr/bin/python

from __future__ import print_function

import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import concatenate
from keras import losses
from keras import regularizers
from keras.constraints import min_max_norm
import h5py

from keras.constraints import Constraint
from keras import backend as K
import numpy as np

#import tensorflow as tf
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.42
#set_session(tf.Session(config=config))

ImportError: cannot import name 'get_config' from 'tensorflow.python.eager.context' (/home/grey/.local/lib/python3.8/site-packages/tensorflow/python/eager/context.py)

In [3]:

def my_crossentropy(y_true, y_pred):
    return K.mean(2 * K.abs(y_true - 0.5) * K.binary_crossentropy(y_pred, y_true), axis=-1)


def mymask(y_true):
    return K.minimum(y_true + 1., 1.)


# def msse(y_true, y_pred):
#     return K.mean(mymask(y_true) * K.square(K.sqrt(y_pred) - K.sqrt(y_true)), axis=-1)

def mmse(y_true, y_pred):
    return K.mean(K.sqrt(y_true) - K.sqrt(y_pred), axis=-1)


def mycost(y_true, y_pred):
    return K.mean(mymask(y_true) * (10 * K.square(K.square(K.sqrt(y_pred) - K.sqrt(y_true))) + K.square(
        K.sqrt(y_pred) - K.sqrt(y_true)) + 0.01 * K.binary_crossentropy(y_pred, y_true)), axis=-1)


def my_accuracy(y_true, y_pred):
    return K.mean(2 * K.abs(y_true - 0.5) * K.equal(y_true, K.round(y_pred)), axis=-1)


class WeightClip(Constraint):
    '''Clips the weights incident to each hidden unit to be inside a range
    '''

    def __init__(self, c=2):
        self.c = c

    def __call__(self, p):
        return K.clip(p, -self.c, self.c)

    def get_config(self):
        return {'name': self.__class__.__name__,
                'c': self.c}


reg = 0.000001
constraint = WeightClip(0.499)

In [27]:
print('Build model...')

main_input = Input(shape=(None, 42), name='main_input')

dense_1 = Dense(24, activation='tanh', name='input_dense', kernel_constraint=constraint, bias_constraint=constraint)(
    main_input)

gru_1 = GRU(24, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, name='gru_1',
            kernel_regularizer=regularizers.l2(reg), recurrent_regularizer=regularizers.l2(reg),
            kernel_constraint=constraint, recurrent_constraint=constraint, bias_constraint=constraint)(dense_1)

noise_input = keras.layers.concatenate([dense_1, gru_1, main_input])

gru_2 = GRU(48, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, name='gru_2',
            kernel_regularizer=regularizers.l2(reg), recurrent_regularizer=regularizers.l2(reg),
            kernel_constraint=constraint, recurrent_constraint=constraint, bias_constraint=constraint)(noise_input)

denoise_input = keras.layers.concatenate([gru_1, gru_2, main_input])

gru_3 = GRU(96, activation='tanh', recurrent_activation='sigmoid', return_sequences=True, name='gru_3',
            kernel_regularizer=regularizers.l2(reg), recurrent_regularizer=regularizers.l2(reg),
            kernel_constraint=constraint, recurrent_constraint=constraint, bias_constraint=constraint)(denoise_input)

dense_2 = Dense(22, activation='sigmoid', name='dense_2', kernel_constraint=constraint, bias_constraint=constraint)(
    gru_3)

final_output = Dense(22, activation='softmax', name='final_output', kernel_constraint=constraint,
                     bias_constraint=constraint)(dense_2)

model = Model(inputs=main_input, outputs=final_output)

model.compile(loss=[mycost, my_crossentropy],
              metrics=[mmse],
              optimizer='adam', loss_weights=[10, 0.5])


model.summary()  # to get details of layers and parameters

# opt = tf.keras.optimizers.Adam(lr=learning_rate, decay=decay_) if required to set particular learning rates




Build model...
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, None, 42)]   0                                            
__________________________________________________________________________________________________
input_dense (Dense)             (None, None, 24)     1032        main_input[0][0]                 
__________________________________________________________________________________________________
gru_1 (GRU)                     (None, None, 24)     3600        input_dense[0][0]                
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, None, 90)     0           input_dense[0][0]                
                                                                 gru_1[0][0] 

In [4]:
batch_size = 32 

print("Loading From File")

filename = "feature_dataset.npz"

with np.load(filename) as data:
    speech_features = data["speech_features"]
    gains = data["gains"]
    # print(np.max(gains))

    # gains = np.clip(gains, 0, 1)
    # print(np.max(gains))

    # Reshape (if Reqd)
    speech_features = speech_features.transpose()
    gains = gains.transpose()

    print(speech_features.shape, gains.shape)

    # x_train, x_test, y_train, y_test = train_test_split(speech_features, gains,
    #                                                     test_size=0.3)

    # x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train,
    #                                                                 test_size=0.2)

    window_size = 2000
    number_of_sequences = int(len(speech_features)/window_size)

    x = speech_features[:number_of_sequences*window_size]
    y = gains[:number_of_sequences*window_size]

    print(x.shape, y.shape)

    x_train = np.reshape(x, (number_of_sequences, window_size, x.shape[1]))
    y_train = np.reshape(y, (number_of_sequences, window_size, y.shape[1]))

    print(x_train.shape, y_train.shape)


Loading From File
(152154, 42) (152154, 22)
(152000, 42) (152000, 22)
(76, 2000, 42) (76, 2000, 22)


In [28]:
print("Train Model")
try:
    model.fit(x_train, y_train, batch_size=batch_size, epochs=20, validation_split=0.1)
except KeyboardInterrupt:
    print("Training Interrupted")

model.save("weights.npz")

Train Model
Epoch 1/20


InternalError:  Blas GEMM launch failed : a.shape=(64000, 42), b.shape=(42, 24), m=64000, n=24, k=42
	 [[node model_2/input_dense/Tensordot/MatMul (defined at <ipython-input-28-b3ecb82e8382>:3) ]] [Op:__inference_train_function_11908]

Function call stack:
train_function


In [5]:
score = model.evaluate(x_train, y_train, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


NameError: name 'model' is not defined

Default GPU Device: /device:GPU:0
